## 1. 무신사 코디맵 크롤링
<br>
- 무신사 코디맵 페이지 (댓글 많은 순) : 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
<br>

- 크롤링할 정보

    - codimap_category : 코디 스타일 구분 (캐주얼, 댄디, 스트릿 ...)
    - codimap_title : 코디 이름
    - codimap_date : 등록 날짜
    - views : 조회수
    - comment_numbers : 댓글 갯수 

### 1) 관련 라이브러리, 모듈 임포트

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

### 2) url 접속 후 페이지 소스 가져오기

In [80]:
import requests
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
res = requests.get(url, headers = {'User-Agent':'Mozilla/5.0'})
soup = BeautifulSoup(res.text, 'html.parser')

### 3) BeautifulSoup을 활용해 파싱한 html에서 요소 긁어내기

1) 페이지 내 li 요소 저장 (total) 
<br></br>
2) 반복문을 활용해 각 li 태그에 접근 
<br></br>
3) li 내에서 필요한 요소 크롤링. 
    - 이 때, codimap, views, comment_numbers는 동일 태그를 공유하므로 한 번에 크롤링 후 인덱싱

In [87]:
result = {'codimap_category' : [], 
          'codimap_title' : [],
          'codimap_date': [],
          'views': [],
          'comment_numbers' : []}

#페이지 내 li section 모두 긁어오기
total = soup.find_all('li',class_ = 'style-list-item')

#각 li section에 접근해 내부 요소 크롤링 
for content in total:
    
    ##codimap_category
    cat = content.find('span', class_='style-list-information__text')
    if cat:
        result['codimap_category'].append(cat.text)
    
    #codimap_title
    title = content.find('strong', class_='style-list-information__title')
    if title:
        result['codimap_title'].append(title.text)
    
    #codimap_date, views, comment_numbers
    post_info = content.find_all('span', class_='post-information__text')

    if post_info : 
        result['codimap_date'].append(post_info[0].text)
        result['views'].append(post_info[1].text.split()[1])
        result['comment_numbers'].append(post_info[2].text.split()[1])
    

### 4) 결과를 데이터프레임 형태로 변환하기

In [89]:
codimap = pd.DataFrame(result)
codimap

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"17,271",50
1,캐주얼,블랙 매니아,21.02.23,"36,329",47
2,캐주얼,오늘의 코디,22.03.22,"13,403",44
3,댄디,추천 댄디 룩,21.09.15,"21,597",32
4,캐주얼,릴랙스 룩,21.02.23,"24,030",25
5,스트릿,힙하다 힙해,21.02.03,"36,085",24
6,댄디,데이트 코디,21.12.24,"10,972",19
7,캐주얼,심플한 코디,21.07.30,"41,354",18
8,댄디,훈훈 남친 룩,21.03.24,"20,001",17
9,댄디,서머 데이트 룩,21.07.26,"8,365",17


### 5) csv 파일로 저장

In [92]:
codimap.to_csv('Ybigta_0105 크롤링_임세은.csv',encoding = 'utf-8-sig')

<br><br/>

## 2. Challenge 과제

<br>
- 무신사 코디맵 페이지 (댓글 많은 순) : 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
<br>

- 크롤링할 정보

    - codimap_category : 코디 스타일 구분 (캐주얼, 댄디, 스트릿 ...)
    - codimap_title : 코디 이름
    - codimap_date : 등록 날짜
    - views : 조회수
    - comment_numbers : 댓글 갯수 
    - codimap_explain : 코디맵 설명 
    - codimap_hashtag : 코디맵 관련 해시태그
    - codimap_imgurl : 이미지 src 링크

### 1) 크롤링

#### 1) 드라이버 호출

#### 2) format을 사용해 페이지 번호를 포함한 url 전달

#### 3) 상품 목록을 보여주는 기본 페이지에서 상품별 카테고리 크롤링
- 기본적으로 각 상세 페이지 내에서 필요한 정보를 크롤링하는 구조로 설계했으나, 상품별 카테고리의 경우 상세 페이지에 나타나지 않기 때문에 전체 화면에서 먼저 크롤링

#### 4) click 매서드를 사용해 각 상품의 상세 페이지로 접근

#### 5) 상세 페이지 url을 불러온 뒤 BeautifulSoup 이용해 파싱 

#### 6) 파싱한 데이터에서 필요 요소 크롤링

#### 7) driver.back으로 전체 페이지 이동 -> 다음 전체 페이지로 넘어가기

In [10]:
result = {'codimap_category' : [], 
          'codimap_title' : [],
          'codimap_date': [],
          'views': [],
          'comment_numbers' : [],
          'codimap_explain': [],
          'codimap_hashtag': [],
          'codimap_imgurl' : []}

#드라이버 지정 
chromedriver = 'chromedriver.exe'
driver = webdriver.Chrome(service = Service(chromedriver))

#page = {} 사용해 페이지 번호 지정
for i in range(5):
    url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page={}'.format(i+1)
    driver.get(url)
    driver.maximize_window()
    driver.implicitly_wait(2)
    wait = WebDriverWait(driver,20)
    
    #codimap_category : 전체 페이지에서 상품별 카테고리 크롤링
    cat = driver.find_elements(By.CSS_SELECTOR,'div.style-list-information > a > span')
    for txt in cat:
        result['codimap_category'].append(txt.text)
    
    #클릭할 element 지정 위해 li section 갯수 받아오기
    num = len(driver.find_elements(By.XPATH,'/html/body/div[3]/div[2]/form/div[4]/div/ul/li'))

    for j in range(num):
        
        # 각 상품을 클릭해 상세페이지로 이동 
        try: 
            page = driver.find_element(By.CSS_SELECTOR,'form > div.right_contents.hover_box > div > ul > li:nth-child({})> div.style-list-item__thumbnail'.format(j+1))
            wait.until(EC.element_to_be_clickable(page))
            page.click()
  
        except:
            page.send_keys('\n')
            page.submit()
        
        # 상세페이지 이동 후 해당 페이지 url 받아오기
        urlnow = driver.current_url
        
        # 받은 url 활용해 파싱 
        res = requests.get(urlnow, headers = {'User-Agent':'Mozilla/5.0'})
        soup = BeautifulSoup(res.text, 'html.parser')

        #codimap_title
        title = soup.select_one('#style_info > h2').text
        if title:
            result['codimap_title'].append(title)

        #codimap_date, views, comment_numbers
        info = soup.find('p',class_='styling_date').text.split('|')

        result['codimap_date'].append(info[0].rstrip()) #codimap_date
        result['views'].append(info[1].split()[1]) #views
        result['comment_numbers'].append(info[2].split()[1]) #comment_numbers
        
        #explain
        explain = soup.select_one('#style_info > div.styling_tag > p').text
        if explain:
            result['codimap_explain'].append(explain)
        #hahtag
        hashtags = soup.select_one('#style_info > div.styling_tag > div').text.split('\n')[1:-1]
        if hashtags:
            result['codimap_hashtag'].append(','.join(hashtags))
        
        #img
        img = 'https://'+ soup.select_one('#style_info > div.codimap-cont > img')['src']
        if img:
            result['codimap_imgurl'].append(img)
        
        # 이전 페이지 = 전체 페이지로 되돌아가기 
        driver.back()

driver.close()

### 2) result를 데이터프레임으로 변환

In [11]:
challenge = pd.DataFrame(result)
challenge

,codimap_category,codimap_title,codimap_date,views,comment_numbers,codimap_explain,codimap_hashtag,codimap_imgurl
0,캐주얼,톤온톤 코디,2021.07.02,"17,316",50,편하게 입기 좋은 반팔 티셔츠와 와이드 팬츠를 연출하고 크로스 백으로 마무리한 캐주얼 룩,"#톤온톤,#뉴트럴톤,#캠퍼스,#피크닉,#와이드핏,#오버핏,#꾸안꾸,#여름,#캐주얼,...",https:////image.msscdn.net/images/codimap/deta...
1,캐주얼,블랙 매니아,2021.02.23,"36,355",47,편안한 무드의 조거 팬츠에 블랙 스웨트셔츠를 매치한 후 볼캡으로 마무리한 캐주얼 룩,"#봄,#캐주얼,#라퍼지스토어,#다이아몬드 레이라,#나이키,#엠프렌즈,#모티패스트,#...",https:////image.msscdn.net/images/codimap/deta...
2,캐주얼,오늘의 코디,2022.03.22,"13,425",44,로고 디테일이 돋보이는 스웨트셔츠와 데님 팬츠를 코디하고 스니커즈로 완성한 캐주얼 룩,"#로고플레이,#오버핏,#스티치,#캠퍼스,#봄,#캐주얼,#유니폼브릿지,#아디다스,#4...",https:////image.msscdn.net/images/codimap/deta...
3,댄디,추천 댄디 룩,2021.09.15,"21,620",32,담백한 디테일의 니트와 셔츠를 레이어드하고 데님 팬츠를 더해 완성한 댄디 룩,"#데이트,#그래픽,#레이어드룩,#가을,#댄디,#엘무드,#뮤지엄바이비컨,#테이크이지,...",https:////image.msscdn.net/images/codimap/deta...
4,캐주얼,릴랙스 룩,2021.02.23,"24,051",25,플라워 자수가 눈에 띄는 니트에 조거 팬츠를 더한 후 레드 볼캡으로 컬러 포인트를 ...,"#봄,#캐주얼,#카시오,#코닥,#제로,#와릿이즌,#뉴발란스,#커버낫,#디지털,#미니...",https:////image.msscdn.net/images/codimap/deta...
...,...,...,...,...,...,...,...,...
295,댄디,데일리 댄디 룩,2021.05.25,"3,558",3,네이비 컬러 반팔 셔츠에 코튼 팬츠를 더하고 스니커즈로 마무리한 댄디 룩,"#데이트,#미니멀,#봄,#댄디,#카시오,#아디다스,#로파이,#에잇세컨즈,#쿼츠 아날...",https:////image.msscdn.net/images/codimap/deta...
296,스트릿,유니크하게,2022.07.27,"1,945",3,그래픽이 매력적인 반소매 티셔츠와 데님 팬츠를 코디하고 크로스 백으로 연출한 스트릿 룩,"#그래픽,#워싱,#와이드핏,#체인,#유니크,#힙합,#여름,#스트릿,#돈애스크마이플랜...",https:////image.msscdn.net/images/codimap/deta...
297,스트릿,원 마일 웨어에 빠져,2020.10.27,"8,851",3,스포티한 매력의 조거 팬츠에 로고 후드 티셔츠를 매치한 후 브라운 비니로 포인트를 ...,"#가을,#스트릿,#칼하트,#제멋,#기프트오리지널,#비니,#셔츠/블라우스,#반소매 티...",https:////image.msscdn.net/images/codimap/deta...
298,아메카지,벌룬 핏의 매력,2022.10.12,"3,638",3,아웃 포켓 디테일이 매력적인 헌팅 재킷과 벌룬 핏 팬츠를 코디하고 스니커즈로 마무리...,"#아메카지,#뉴트럴톤,#벌룬핏,#헨리넥,#가을,#아메리칸 캐주얼,#파이살론,#리프로...",https:////image.msscdn.net/images/codimap/deta...


### 3) csv로 저장

In [8]:
challenge.to_csv('Ybigta_0105 크롤링 challenge_임세은.csv',encoding = 'utf-8-sig')